In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
from tensorflow.keras import layers
import kaggle
import os

In [11]:
kaggle.api.authenticate()
kaggle.api.dataset_download_files('samithsachidanandan/human-face-emotions', path='./Data', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/samithsachidanandan/human-face-emotions


In [12]:
data = "./Data"
faces = os.listdir(data)

for images in faces:
  face_list = os.listdir(data + "/" + images)
  len_face_list = len(face_list)
  print(f"{images} have {len_face_list}")

Happy have 18439
Sad have 12553
Fear have 9732
Angry have 10148
Suprise have 8227


In [13]:
train = tf.keras.utils.image_dataset_from_directory(
    directory = data,
    seed = 32,
    validation_split = 0.2,
    subset = "training",
    batch_size = 32,
    image_size = (120,120)
)

val = tf.keras.utils.image_dataset_from_directory(
    directory = data,
    seed = 32,
    validation_split = 0.2,
    subset = "validation",
    batch_size = 32,
    image_size = (120,120)
)

Found 59099 files belonging to 5 classes.
Using 47280 files for training.
Found 59099 files belonging to 5 classes.
Using 11819 files for validation.


In [14]:

class_names = train.class_names
print(class_names)

['Angry', 'Fear', 'Happy', 'Sad', 'Suprise']


In [15]:
normalization_layer = layers.Rescaling(1./255)
normalized_train = train.map(lambda x, y: (normalization_layer(x), y))
normalized_val = val.map(lambda x, y: (normalization_layer(x), y))

In [16]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(120,120,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
])

/Users/lukassomwong/Developer/VulnerabilitySeverityPredictor/venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 118, 118, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 59, 59, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 57, 57, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 26, 26, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,769,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,862,917 (10.92 MB)

 Trainable params: 2,862,917 (10.92 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
history = model.fit(
    normalized_train,
    validation_data = normalized_val,
    epochs = 10
)

Epoch 1/10
 639/1478 ━━━━━━━━━━━━━━━━━━━━ 1:21 97ms/step - accuracy: 0.3675 - loss: 1.5064

KeyboardInterrupt: 

In [ ]:
model.save("emotion_detection_model.h5")

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(10)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save("emotion_detection_model.h5")

In [ ]:
from tensorflow.keras.models import load_model
loaded_model = load_model("emotion_detection_model.h5")

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
def prepare_image(img_path):
    img = image.load_img(img_path, target_size=(120, 120))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

In [ ]:
test_image_path = "/kaggle/input/human-face-emotions/Data/happy/happy1.jpg"
img_array = prepare_image(test_image_path)
predictions = loaded_model.predict(img_array)
predicted_class = class_names[np.argmax(predictions)]
print(f"The predicted emotion is: {predicted_class}")